In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
address = "https://raw.githubusercontent.com/Jaeik-Jeong/STCNN/main/data/"
data_csv = pd.read_csv(address+'ny_data_GAA.csv', index_col=0)

In [ ]:
# LSTM

M = 18 # The number of past data for input
H = 6 # The number of future data for output
N = 67 # The number of sites

n_layers       = 2
in_size        = N
sequence_size  = M
hidden_size    = N*H*2
out_size       = N*H

LSTM_learning_rate = 1e-4
LSTM_epoch = 100

class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.rnn = nn.LSTM(in_size, hidden_size, n_layers, batch_first=True)
        self.fc_out = nn.Linear(hidden_size, out_size)
        self.optimizer = torch.optim.Adam(self.parameters(), lr=LSTM_learning_rate)

    def forward(self, x):
        x, _ = self.rnn(x)
        out = self.fc_out(x[:,-1,:])
        out = out.view(-1, out_size)
        return out
        
    def train_net(self, x, y):
        x, y = torch.tensor(x,dtype=torch.float), torch.tensor(y,dtype=torch.float)
        loss = F.mse_loss(self.forward(x), y)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

In [ ]:
# LSTM training

data = np.array(data_csv)
data_size = np.size(data, 1)

train_data_size  = int(data_size*0.6)
train_batch_size = train_data_size - M - H + 1

batch_size = 128
total_batch = int((train_batch_size-1)/batch_size) + 1

train_input  = np.zeros((train_batch_size, M, N))
train_output = np.zeros((train_batch_size, N*H))
for i in range(train_batch_size):
    train_input[i,:]  = np.transpose(data[:,i:i+M])
    train_output[i,:] = np.reshape(data[:,i+M:i+M+H], (N*H))


val_data_size    = int(data_size*0.8)
val_batch_size   = val_data_size - train_data_size - M - H + 1

val_input  = np.zeros((val_batch_size, M, N))
val_output = np.zeros((val_batch_size, N*H))
for i in range(val_batch_size):
    val_input[i,:]  = np.transpose(data[:,train_data_size+i:train_data_size+i+M])
    val_output[i,:] = np.reshape(data[:,train_data_size+i+M:train_data_size+i+M+H], (N*H))


model = LSTM()
mse_train, mse_val = [], [] # Mean Squared Error
mae_train, mae_val = [], [] # Mean Absolute Error
for epoch in range(LSTM_epoch):
    for i in range(total_batch):
        batch_x = train_input[batch_size*i:batch_size*(i+1)]
        batch_y = train_output[batch_size*i:batch_size*(i+1)]
        model.train_net(batch_x, batch_y)
    
    train_predict = model.forward(torch.tensor(train_input, dtype=torch.float)).detach().numpy()
    mse_train = np.mean(np.square(train_predict - train_output))
    mae_train = np.mean(np.abs(train_predict - train_output))
    scale_train = np.mean(np.abs(train_output[1:] - train_output[:-1]))
    
    val_predict = model.forward(torch.tensor(val_input, dtype=torch.float)).detach().numpy()
    mse_val = np.mean(np.square(val_predict - val_output))
    mae_val = np.mean(np.abs(val_predict - val_output))
    scale_val = np.mean(np.abs(val_output[1:] - val_output[:-1]))

    NRMSE_train = round(100*np.sqrt(mse_train),2)
    MAPE_train  = round(100*mae_train,2)
    MASE_train  = round(mae_train/scale_train,2)
    NRMSE_val   = round(100*np.sqrt(mse_val),2)
    MAPE_val    = round(100*mae_val,2)
    MASE_val    = round(mae_val/scale_val,2)

    print("epoch: {}".format(epoch+1))
    print("NRMSE_train: {}%".format(NRMSE_train).ljust(25), end="")
    print("MAPE_train: {}%".format(MAPE_train).ljust(25), end="")
    print("MASE_train: {}".format(MASE_train).ljust(25))
    print("NRMSE_val: {}%".format(NRMSE_val).ljust(25), end="")
    print("MAPE_val: {}%".format(MAPE_val).ljust(25), end="")
    print("MASE_val: {}".format(MASE_val).ljust(25))
    print("------------------------------------------------------------------------------------------")

In [ ]:
# LSTM test

test_batch_size   = data_size - val_data_size - M - H + 1

test_input  = np.zeros((test_batch_size, M, N))
test_output = np.zeros((test_batch_size, N, H))
for i in range(test_batch_size):
    test_input[i,:]  = np.transpose(data[:,val_data_size+i:val_data_size+i+M])
    test_output[i,:] = np.reshape(data[:,val_data_size+i+M:val_data_size+i+M+H], (N, H))

test_predict = model.forward(torch.tensor(test_input, dtype=torch.float)).detach().numpy()
test_predict = np.reshape(test_predict, (-1, N, H))
mse_test = np.mean(np.square(test_predict - test_output))
mae_test = np.mean(np.abs(test_predict - test_output))
scale_test = np.mean(np.abs(test_output[1:] - test_output[:-1]))

NRMSE_test = round(100*np.sqrt(mse_test),2)
MAPE_test  = round(100*mae_test,2)
MASE_test  = round(mae_test/scale_test,2)

print("NRMSE_test: {}%".format(NRMSE_test).ljust(25), end="")
print("MAPE_test: {}%".format(MAPE_test).ljust(25), end="")
print("MASE_test: {}".format(MASE_test).ljust(25))

test_output_a = np.mean(test_output, axis=1)
test_predict_a = np.mean(test_predict, axis=1)
mae_test_a = np.mean(np.abs(test_predict_a - test_output_a))
scale_test_a = np.mean(np.abs(test_output_a[1:] - test_output_a[:-1]))

MAPE_test_a      = round(100*mae_test_a,2)
MAPE_improvement = round(100*(MAPE_test-MAPE_test_a)/MAPE_test,2)

print("MAPE_test (Aggregation): {}%".format(MAPE_test_a).ljust(36), end="")
print("MAPE_test Improvement: {}%".format(MAPE_improvement).ljust(36))